In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("DataFrameBasic")

from pyspark.sql import SparkSession
spark =SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc= spark.sparkContext

22/03/03 22:04:43 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/03 22:04:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/03 22:04:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/03 22:04:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/03 22:04:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [30]:
products = [
    (1, 'iphone', 1000.0, 100, 0),
    (2, 'Galaxy', 545.50, 101, None),
    (3, 'Pixel', 645.99, 101, None)
]

schema = ['product_id', 'product_name', 'price', 'brand_id', 'offer']

productDf = spark.createDataFrame(data = products, schema = schema)
productDf.printSchema()
productDf.show()

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- brand_id: long (nullable = true)
 |-- offer: long (nullable = true)

+----------+------------+------+--------+-----+
|product_id|product_name| price|brand_id|offer|
+----------+------------+------+--------+-----+
|         1|      iphone|1000.0|     100|    0|
|         2|      Galaxy| 545.5|     101| null|
|         3|       Pixel|645.99|     101| null|
+----------+------------+------+--------+-----+



In [5]:
productDf.rdd.getNumPartitions()

1

In [7]:
df = productDf.filter (productDf['price'] <= 750)

In [8]:
df.show()

+----------+------------+------+--------+
|product_id|product_name| price|brand_id|
+----------+------------+------+--------+
|         2|      Galaxy| 545.5|     101|
|         3|       Pixel|645.99|     101|
+----------+------------+------+--------+



In [9]:
df=productDf.select( "product_name", 'price')
df.printSchema()
df.show()

root
 |-- product_name: string (nullable = true)
 |-- price: double (nullable = true)

+------------+------+
|product_name| price|
+------------+------+
|      iphone|1000.0|
|      Galaxy| 545.5|
|       Pixel|645.99|
+------------+------+



In [11]:
df = productDf.selectExpr ('product_name', 'upper(product_name)',
                           'price', 'price * .9 as grand_total')

df.printSchema()
df.show()

root
 |-- product_name: string (nullable = true)
 |-- upper(product_name): string (nullable = true)
 |-- price: double (nullable = true)
 |-- grand_total: double (nullable = true)

+------------+-------------------+------+-----------------+
|product_name|upper(product_name)| price|      grand_total|
+------------+-------------------+------+-----------------+
|      iphone|             IPHONE|1000.0|            900.0|
|      Galaxy|             GALAXY| 545.5|           490.95|
|       Pixel|              PIXEL|645.99|581.3910000000001|
+------------+-------------------+------+-----------------+



In [12]:
df = productDf.selectExpr ('product_name', 'upper(product_name) as title',
                           'price', 'price * .9 as grand_total')

df.printSchema()
df.show()

root
 |-- product_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- price: double (nullable = true)
 |-- grand_total: double (nullable = true)

+------------+------+------+-----------------+
|product_name| title| price|      grand_total|
+------------+------+------+-----------------+
|      iphone|IPHONE|1000.0|            900.0|
|      Galaxy|GALAXY| 545.5|           490.95|
|       Pixel| PIXEL|645.99|581.3910000000001|
+------------+------+------+-----------------+



In [ ]:
df = productDf.withColumn('offer_price'

In [14]:
df = productDf.withColumnRenamed('price', 'total')
df.printSchema()
df.show()

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- total: double (nullable = true)
 |-- brand_id: long (nullable = true)

+----------+------------+------+--------+
|product_id|product_name| total|brand_id|
+----------+------------+------+--------+
|         1|      iphone|1000.0|     100|
|         2|      Galaxy| 545.5|     101|
|         3|       Pixel|645.99|     101|
+----------+------------+------+--------+



In [16]:
df = productDf.drop('brand_id')
df.printSchema()
df.show()

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: double (nullable = true)

+----------+------------+------+
|product_id|product_name| price|
+----------+------------+------+
|         1|      iphone|1000.0|
|         2|      Galaxy| 545.5|
|         3|       Pixel|645.99|
+----------+------------+------+



In [21]:
df = productDf.filter (( productDf.price >= 500) & (productDf ['price'] <600))
df.printSchema()
df.show()

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- brand_id: long (nullable = true)

+----------+------------+-----+--------+
|product_id|product_name|price|brand_id|
+----------+------------+-----+--------+
|         2|      Galaxy|545.5|     101|
+----------+------------+-----+--------+



In [22]:
df = productDf.where (( productDf.price >= 500) & (productDf ['price'] <600))
df.printSchema()
df.show()

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- brand_id: long (nullable = true)

+----------+------------+-----+--------+
|product_id|product_name|price|brand_id|
+----------+------------+-----+--------+
|         2|      Galaxy|545.5|     101|
+----------+------------+-----+--------+



In [23]:
df = productDf.where ('price >= 500 AND price <600')
df.printSchema()
df.show()

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- brand_id: long (nullable = true)

+----------+------------+-----+--------+
|product_id|product_name|price|brand_id|
+----------+------------+-----+--------+
|         2|      Galaxy|545.5|     101|
+----------+------------+-----+--------+



In [24]:
print(productDf.price)
print(productDf['price'])

from pyspark.sql.functions import col
print(col('price'))

Column<b'price'>
Column<b'price'>
Column<b'price'>


In [25]:
from pyspark.sql.functions import lit
df = productDf.withColumn('qty', lit(4))\
            .withColumn('amount', col('qty') * ('price'))

df.printSchema()
df.show()

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- brand_id: long (nullable = true)
 |-- qty: integer (nullable = false)
 |-- amount: double (nullable = true)

+----------+------------+------+--------+---+------+
|product_id|product_name| price|brand_id|qty|amount|
+----------+------------+------+--------+---+------+
|         1|      iphone|1000.0|     100|  4|  null|
|         2|      Galaxy| 545.5|     101|  4|  null|
|         3|       Pixel|645.99|     101|  4|  null|
+----------+------------+------+--------+---+------+



In [26]:
df = productDf.sort('price')
df.show() 

+----------+------------+------+--------+
|product_id|product_name| price|brand_id|
+----------+------------+------+--------+
|         2|      Galaxy| 545.5|     101|
|         3|       Pixel|645.99|     101|
|         1|      iphone|1000.0|     100|
+----------+------------+------+--------+



In [28]:
from pyspark.sql.functions import desc
df = productDf.sort(desc('price'))
df.show()                         

+----------+------------+------+--------+
|product_id|product_name| price|brand_id|
+----------+------------+------+--------+
|         1|      iphone|1000.0|     100|
|         3|       Pixel|645.99|     101|
|         2|      Galaxy| 545.5|     101|
+----------+------------+------+--------+



In [29]:
df = productDf.sort (productDf.price.asc())
df.show()
df = productDf.sort (productDf.price.desc())
df.show()

+----------+------------+------+--------+
|product_id|product_name| price|brand_id|
+----------+------------+------+--------+
|         2|      Galaxy| 545.5|     101|
|         3|       Pixel|645.99|     101|
|         1|      iphone|1000.0|     100|
+----------+------------+------+--------+

+----------+------------+------+--------+
|product_id|product_name| price|brand_id|
+----------+------------+------+--------+
|         1|      iphone|1000.0|     100|
|         3|       Pixel|645.99|     101|
|         2|      Galaxy| 545.5|     101|
+----------+------------+------+--------+



In [33]:
productDf.show()
df= productDf.fillna(value = 0, subset =['offer'])
df.show()

+----------+------------+------+--------+-----+
|product_id|product_name| price|brand_id|offer|
+----------+------------+------+--------+-----+
|         1|      iphone|1000.0|     100|    0|
|         2|      Galaxy| 545.5|     101| null|
|         3|       Pixel|645.99|     101| null|
+----------+------------+------+--------+-----+

+----------+------------+------+--------+-----+
|product_id|product_name| price|brand_id|offer|
+----------+------------+------+--------+-----+
|         1|      iphone|1000.0|     100|    0|
|         2|      Galaxy| 545.5|     101|    0|
|         3|       Pixel|645.99|     101|    0|
+----------+------------+------+--------+-----+

